In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import argparse
import psutil
import numpy as np
import logging
import numpy as np
import tensorflow as tf
import time
import glob
from tensorflow.python.framework import ops
import math
from dotenv import find_dotenv, load_dotenv
import ftplib
from matplotlib import pyplot as plt
from collections import Counter

from scipy.stats.stats import pearsonr

nb_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'data')
if nb_dir not in sys.path:
    sys.path.insert(0, nb_dir)
print(sys.path)
import features_selection as fs
import make_dataset as md
import build_data as bd
import get_raw_data as grd

models_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'models')
if models_dir not in sys.path:
    sys.path.insert(0, models_dir)
import nn_real as nn

load_dotenv(find_dotenv())

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.set_option('io.hdf.default_format','table')

C:\Users\sandr\AppData\Local\conda\conda\envs\tensorflowenvironment\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['C:\\Users\\sandr\\Documents\\GitHub\\MLMortgage\\src\\data', '', 'C:\\Users\\sandr\\AppData\\Local\\conda\\conda\\envs\\tensorflowenvironment\\python36.zip', 'C:\\Users\\sandr\\AppData\\Local\\conda\\conda\\envs\\tensorflowenvironment\\DLLs', 'C:\\Users\\sandr\\AppData\\Local\\conda\\conda\\envs\\tensorflowenvironment\\lib', 'C:\\Users\\sandr\\AppData\\Local\\conda\\conda\\envs\\tensorflowenvironment', 'C:\\Users\\sandr\\AppData\\Local\\conda\\conda\\envs\\tensorflowenvironment\\lib\\site-packages', 'C:\\Users\\sandr\\AppData\\Local\\conda\\conda\\envs\\tensorflowenvironment\\lib\\site-packages\\Mako-1.0.7-py3.6.egg', 'C:\\Users\\sandr\\AppData\\Local\\conda\\conda\\envs\\tensorflowenvironment\\lib\\site-packages\\win32', 'C:\\Users\\sandr\\AppData\\Local\\conda\\conda\\envs\\tensorflowenvironment\\lib\\site-packages\\win32\\lib', 'C:\\Users\\sandr\\AppData\\Local\\conda\\conda\\envs\\tensorflowenvironment\\lib\\site-packages\\Pythonwin', 'C:\\Users\\sandr\\AppData\\Local\\conda\\con

In [3]:
RAW_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'raw') 
PRO_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'processed')
RANDOM_SEED = 123  # Set the seed to get reproducable results.
DT_FLOAT = np.float32
NP_FLOAT = np.dtype('float32')

train_dir = 'chuncks_random_c1mill' #'chuncks_random_c1mill_slices' #'chuncks_random_c1millx2_train'
valid_dir = 'chuncks_random_c1mill_valid'
test_dir = ''
train_period=[121,323] #[121,279] #[121, 143] 
valid_period=[324,329] #[280,285] #[144, 147] 
test_period=[330,342] #[286,304] #[148, 155]
batch_size=100000

print(RAW_DIR, PRO_DIR)

def get_files_dict(train_dir, valid_dir, test_dir):        
    ext = "*.h5"

    files_dict = {'train': glob.glob(os.path.join(PRO_DIR, train_dir, ext)), 
                  'valid': glob.glob(os.path.join(PRO_DIR, valid_dir, ext)), 
                  'test': glob.glob(os.path.join(PRO_DIR, test_dir, ext))}

    return files_dict

def architecture_settings(files_dict):
    architecture = {}    
    ok_inputs = True
    
    for key in files_dict.keys():
        total_records = 0
        for file in files_dict[key]:                                
            with pd.HDFStore(file) as dataset_file:
                if (ok_inputs): 
                    index_length = len(dataset_file.get_storer(key+'/features').attrs.data_columns)
                    architecture['n_input'] = dataset_file.get_storer(key+ '/features').ncols - index_length
                    architecture['n_classes'] = dataset_file.get_storer(key+'/labels').ncols - index_length
                    ok_inputs = False                
                total_records += dataset_file.get_storer(key + '/features').nrows
        architecture[key + '_num_examples'] = total_records                            
    
    architecture['total_num_examples'] = architecture['train_num_examples'] # 10000000
    return architecture

#To sum up the dataset per worker (assuming the same size of files per worker approximately):
files_dict = get_files_dict(train_dir, valid_dir, test_dir)
architecture = architecture_settings(files_dict)
print(architecture)

DATA = md.get_h5_data(PRO_DIR, architecture, train_dir, valid_dir, None, train_period=train_period, valid_period=None, test_period=None)

C:\Users\sandr\Documents\GitHub\MLMortgage\data\raw C:\Users\sandr\Documents\GitHub\MLMortgage\data\processed


HDF5ExtError: HDF5 error back trace

  File "C:\bld\hdf5_1519268879329\work\hdf5-1.8.18\src\H5F.c", line 604, in H5Fopen
    unable to open file
  File "C:\bld\hdf5_1519268879329\work\hdf5-1.8.18\src\H5Fint.c", line 1087, in H5F_open
    unable to read superblock
  File "C:\bld\hdf5_1519268879329\work\hdf5-1.8.18\src\H5Fsuper.c", line 294, in H5F_super_read
    unable to load superblock
  File "C:\bld\hdf5_1519268879329\work\hdf5-1.8.18\src\H5AC.c", line 1262, in H5AC_protect
    H5C_protect() failed.
  File "C:\bld\hdf5_1519268879329\work\hdf5-1.8.18\src\H5C.c", line 3574, in H5C_protect
    can't load entry
  File "C:\bld\hdf5_1519268879329\work\hdf5-1.8.18\src\H5C.c", line 7954, in H5C_load_entry
    unable to load entry
  File "C:\bld\hdf5_1519268879329\work\hdf5-1.8.18\src\H5Fsuper_cache.c", line 476, in H5F_sblock_load
    truncated file: eof = 499029954, sblock->base_addr = 0, stored_eoa = 1150129075

End of HDF5 error back trace

Unable to open/create file 'C:\Users\sandr\Docum

In [ ]:
print(DATA.train._dict[0]['nrows'])
print(len(DATA.train.features_list), DATA.train.features_list)
print(len(DATA.train.labels_list), DATA.train.labels_list)

In [ ]:
if (DATA.validation._dict):
    print(DATA.validation._dict[0]['nrows'])

In [ ]:
train_X = np.array(DATA.train._dict[0]['dataset_features'][:3000000]) #df_chunk.values
#del DATA.train._dict[0]['dataset_features']

In [ ]:
train_Y = np.argmax(np.array(DATA.train._dict[0]['dataset_labels'][:3000000]), axis=1) #y.values
#del DATA.train._dict[0]['dataset_labels']
print(train_X.shape)
print(train_Y.shape)
print('Training Dataset shape %s' % Counter(train_Y))

In [ ]:
test_X = np.array(DATA.validation._dict[0]['dataset_features'])
#del DATA.validation._dict[0]['dataset_features']
test_Y = np.argmax(np.array(DATA.validation._dict[0]['dataset_labels']), axis=1)
#del DATA.validation._dict[0]['dataset_labels']
print(test_X.shape)
print(test_Y.shape)
print('Validation Dataset shape %s' % Counter(test_Y))

In [ ]:
import xgboost as xgb


xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
# setup parameters for xgboost

In [ ]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1 # 0.3
param['max_depth'] = 6 #3
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 7
param['tree_method'] = 'gpu_hist' #'gpu_hist' > 'exact' == 'hist' > 'gpu_exact'
# param['n_gpus'] = 1 #kernel died!
param['eval_metric'] = ['mlogloss', 'merror'] # 'auc', 'aucpr',
#param['max_delta_step'] = 1 #by default = 0
num_round = 300

In [ ]:
evals_result = {}
bst = xgb.train(param, xg_train, num_round, watchlist, early_stopping_rounds=10, evals_result=evals_result) #  xgb_model=None, Validation error needs to decrease at least every early_stopping_rounds to continue training

In [ ]:
print(bst)
for e_name, e_mtrs in evals_result.items():
    print('- {}'.format(e_name))
    for e_mtr_name, e_mtr_vals in e_mtrs.items():
        print('   - {}'.format(e_mtr_name))
        print('      - {}'.format(e_mtr_vals))

In [ ]:
print(bst.best_score, bst.best_iteration)
print(bst.best_ntree_limit)
# print(xgb.get_xgb_params()) # get_params this is only for sklearn implementation of XGBoost!

In [ ]:
# get prediction:
# pred = bst.predict(xg_test)
# get the best prediction:
pred = bst.predict(xg_test, ntree_limit=bst.best_ntree_limit)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_Y, pred))

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
results = confusion_matrix(test_Y, pred, labels=[0, 1, 2, 3, 4, 5, 6])
print('Confusion Matrix: ', results)
#F1Score: 
print('F1Score-Macro: ', f1_score(test_Y, pred, average='macro'))

print('F1Score-Micro: ', f1_score(test_Y, pred, average='micro'))

print('F1Score-Weighted: ', f1_score(test_Y, pred, average='weighted'))

print('F1Score-AverageNone: ', f1_score(test_Y, pred, average=None))

In [ ]:
bst.save_model('allNoInvTrans_3mill_25mill_001.model')

In [ ]:
# dump model
#bst.dump_model('dump.raw.txt')
# dump model with feature map
bst.dump_model('allNoInvTrans_3mill_25mill_dump.txt', 'allNoInvTrans_3mill_25mill_featmap.txt')

In [ ]:
#Load the model:
# bst = xgb.Booster({'nthread': 4})  # init model
bst1 = xgb.Booster(model_file='allNoInvTrans_3mill_25mill_001.model')
# bst.load_model('model.bin')  # load data

#testing set:
#dtest = xgb.DMatrix(data)
#ypred = bst.predict(dtest)
# If early stopping is enabled during training:
ypred = bst.predict(xg_test, ntree_limit=bst.best_ntree_limit)

In [ ]:
print(xgb.get_score())

In [ ]:
# to plot the importance of variables:
import matplotlib.pyplot as plt
xgb.plot_importance(bst)

In [ ]:
import graphviz 
# Specifying the ordinal number of the target tree. 
xgb.plot_tree(bst, num_trees=2)

In [ ]:
# plot graphviz object:
xgb.to_graphviz(bst, num_trees=2)